<a href="https://colab.research.google.com/github/toru-matano/Random-Forest-Tensorflow-test/blob/main/random_forest_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 41.3 MB/s eta 0:00:00


In [44]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf

from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing = pd.DataFrame(housing.data, columns=housing.feature_names).assign(price=np.array(housing.target))

def split_dataset(dataset, test_ratio=0.2):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset.iloc[~test_indices], dataset.iloc[test_indices]

train_ds_pd, test_ds_pd = split_dataset(housing)

housing

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [55]:
object_val = "price"
task = 2  # regression
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, task=task, label=object_val, max_num_classes=5000)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, task=task, label=object_val, max_num_classes=5000)

# Hyper-parameters to optimize.
# tuner.discret("max_depth", [4, 5, 6, 7])

model = tfdf.keras.RandomForestModel(task=task)

model.compile(
    loss="MeanSquaredError",
    metrics=["accuracy"])

model.fit(train_ds)

model.summary()

Use /tmp/tmpakh6vjcc as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.309088. Found 16514 examples.
Training model...
Model trained in 0:00:19.814998
Compiling model...
Model compiled.
Model: "random_forest_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (8):
	AveBedrms
	AveOccup
	AveRooms
	HouseAge
	Latitude
	Longitude
	MedInc
	Population

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.     "MedInc"  0.481228 ################
    2.   "AveOccup"  0.216976 #####
    3.   "Latitude"  0.208230 #####
    4.  "Longitude"  0.152898 ##
    5.   "AveRooms"  0.150691 ##
    6.   "HouseAge"  0.114719 #
    7.  "AveBed

In [49]:
mse, accuracy = model.evaluate(test_ds)
np.sqrt(mse), accuracy

5/5 [==============================] - 1s 63ms/step - loss: 0.2504 - accuracy: 0.0022


(0.5004416243972893, 0.0021812894847244024)